In [72]:
import pandas as pd
import lib.excel_util as excel_util

# define useful constants
# 地块类型与季次的映射
landtype_season_map = {
    '平旱地': ['单季'],
    '梯田': ['单季'],
    '山坡地': ['单季'],
    '水浇地': ['单季', '第一季', '第二季'],
    '普通大棚': ['第一季', '第二季'],
    '智慧大棚': ['第一季', '第二季']
}

workbook_path_1 = r'\\wsl.localhost\Ubuntu-22.04\home\flas\source\course\modelling\agristrat\附件1.xlsx'
workbook_path_2 = r'\\wsl.localhost\Ubuntu-22.04\home\flas\source\course\modelling\agristrat\附件2.xlsx'

# Preprocessing
## Tidy Land

In [ ]:
land_data = pd.read_excel("附件1.xlsx").drop(columns=['说明 '])
land_data.replace("普通大棚 ", "普通大棚", inplace=True)
land_data.info()

In [ ]:
excel_util.export_to_excel(land_data, workbook_path_1, 'land')

## Tidy Crop

In [ ]:
crop_data = pd.read_excel("附件1.xlsx", sheet_name="crop_tidied")

# Function to split land and season
def split_land_season(land_season_str):
    pairs = land_season_str.split('|')
    result = []
    for pair in pairs:
        if '-' in pair:
            land, *seasons = pair.split('-')
            for season in seasons:
                result.append(f'{land}-{season}')
        else:
            result.append(f"{pair}-单季")
    return result

# Apply the function to each row and explode the list into separate rows
crop_data['land_season_split'] = crop_data['种植耕地'].apply(split_land_season)
crop_data_exploded = crop_data.explode('land_season_split')

# Create dummy variables
dummies = pd.get_dummies(crop_data_exploded['land_season_split'])

# Group back to original rows by summing the dummy variables
crop_data_dummies = dummies.groupby(crop_data_exploded.index).sum()

# Concatenate the original DataFrame with the dummy variables
crop_data_concat = pd.concat([crop_data, crop_data_dummies], axis=1)

crop_data_concat['id'] = crop_data_concat['作物编号'] - 1

# Drop the intermediate column
crop_data = crop_data_concat[['id', '作物名称', 
         '平旱地-单季', '梯田-单季', '山坡地-单季', 
         '水浇地-单季', '水浇地-第一季', '水浇地-第二季', 
         '普通大棚-第一季', '普通大棚-第二季', 
         '智慧大棚-第一季', '智慧大棚-第二季']]

# Display the final DataFrame
crop_data.info()

In [40]:
excel_util.export_to_excel(crop_data, workbook_path_1, 'crop_dummied')

## Generate Configs

In [ ]:
# join landtype and season
import pandas as pd

# 将地块类型与季次映射转换为 DataFrame
landtype_season_df = pd.DataFrame([
    {'地块类型': land_type, '季次': season}
    for land_type, seasons in landtype_season_map.items()
    for season in seasons
])

# 使用 merge 进行笛卡尔积，生成所有地块类型和季次的组合
land_season = pd.merge(land_data, landtype_season_df, on='地块类型')
land_season['id'] = land_season.index
land_season = land_season[['id', '地块名称', '地块类型', '季次', '地块面积/亩']]

# 查看结果
land_season.info()

In [35]:
excel_util.export_to_excel(land_season, workbook_path_1, 'land_season')

In [ ]:
crop_data.info() # has dummied columns
land_season.info()

In [ ]:
import pandas as pd

# Step 1: Perform a Cartesian product (cross join)
combined_df = pd.merge(crop_data, land_season, how='cross')

# Step 2: Apply the filtering logic
def is_valid_combination(row):
    land_type = row['地块类型']
    season = row['季次']
    
    # Check if the land type and season combination is valid
    if land_type in landtype_season_map and season in landtype_season_map[land_type]:
        column_name = f"{land_type}-{season}"
        return row[column_name] == 1
    return False

# Apply the filtering function to the combined DataFrame
configs = combined_df[combined_df.apply(is_valid_combination, axis=1)].drop(columns=[
    '平旱地-单季', '梯田-单季', '山坡地-单季', 
    '水浇地-单季', '水浇地-第一季', '水浇地-第二季', 
    '普通大棚-第一季', '普通大棚-第二季', 
    '智慧大棚-第一季', '智慧大棚-第二季', '地块面积/亩'
]).rename(columns={'id_x':'id_crop', 'id_y':'id_landseason'})[['id_crop', 'id_landseason', '作物名称', '地块类型', '季次']]

configs

## Tidy Price

In [ ]:
price_data = pd.read_excel("附件2.xlsx", sheet_name="price")
price_data['id_crop'] = price_data['作物编号'] - 1
price_data.rename(columns={'种植季次':'季次'}, inplace=True)
price_data.replace("普通大棚 ", "普通大棚", inplace=True)
price_data

In [ ]:
print(pd.merge.__doc__)

In [ ]:
config_w_profit = pd.merge(configs, price_data[['id_crop', '季次', '地块类型', 'profit']], on=['id_crop', '地块类型', '季次'], how='left')
config_w_profit

# Define Model

In [ ]:
import gurobipy as gp
from gurobipy import GRB

try:
    m = gp.Model("mip1")

    # (year, landseason, crop)
    x = m.addMVar((7, len(land_season), len(crop_data)), lb=0, vtype=GRB.CONTINUOUS, name="x")

    # m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # m.addConstr(x + y >= 1, "c1")

    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")